In [1]:
# NEXT STEPS:
# 1. Test with >2 different sequence inputs
# 2. Modularise code

# add note to ignore warnings

In [2]:
import numpy as np
import Bio
import VCF

In [3]:
files = ['../data/vcf/person_1/FR07961007.pass.recode.vcf',
        '../data/vcf/ERZ389531/FR07961004.pass.recode.vcf']
# ideally, load these from a statis storage file?

num_people = len(files)

In [4]:
# Define list of chromosomes
chromosomes = list(np.arange(1,23,1))
chromosomes.append('X')
chromosomes.append('Y')

In [5]:
# Create list of dataframes (with all info)
full_dfs = []

for file in files:
    full_dfs.append(VCF.dataframe(file))

/Users/chrislovejoy/opt/miniconda3/envs/Bioinf_assignments/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3338: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [6]:
# Visualise first df
full_dfs[0]

,CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO
0,1,54676,.,C,T,556.77,PASS,.
1,1,54708,.,G,C,556.77,PASS,.
2,1,54710,.,A,ATTTTT,194.73,PASS,.
3,1,61987,.,A,G,814.77,PASS,.
4,1,61989,.,G,C,814.77,PASS,.
...,...,...,...,...,...,...,...,...
4137171,GL000192.1,473530,.,C,T,1094.73,PASS,.
4137172,GL000192.1,476592,.,T,C,1358.77,PASS,.
4137173,GL000192.1,479530,.,G,A,1609.77,PASS,.
4137174,GL000192.1,485280,.,GT,G,118.73,PASS,.


In [7]:
# temporarily trim each df (for development)
for index, df in enumerate(full_dfs):
    full_dfs[index] = df.loc[df.index < 10000]


In [8]:
# Add column for position and alternative variant
def combine(row):
    return str(row['POS']) + row['ALT']

for df in full_dfs:
    df['POS-ALT'] = df.apply(lambda row: combine(row), axis=1)

<ipython-input-8-7c9f3937f92c>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['POS-ALT'] = df.apply(lambda row: combine(row), axis=1)


In [9]:
# Create list of genotype variants on each chromosome
genotype_variations = []

for df in full_dfs:
    variants = {}
    for chromosome in chromosomes:
        variants[chromosome] = list(df.loc[df.CHROM == chromosome]['POS-ALT'])
    genotype_variations.append(variants)


In [10]:
# Create empty similarity matrix
similarity_matrix = np.zeros([num_people,num_people])
similarity_matrix

array([[0., 0.],
       [0., 0.]])

In [11]:
def count_total_variants(person_variants):
    variants_count = 0
    for chromosome in chromosomes:
        variants_count += len(person_variants[chromosome])
    return variants_count

In [12]:
def count_similar_variants(person1_variants, person2_variants):
    """
    Input: variants, which is a dictionary of chrosomes mapped to lists of variants 
    """
    overlapping_variants = 0
    for chromosome in chromosomes:
        list1 = person1_variants[chromosome]
        list2 = person2_variants[chromosome]
        for variant in list1:
            if variant in list2:
                overlapping_variants += 1
    return overlapping_variants

In [13]:
for index1, person1 in enumerate(files):
    for index2, person2 in enumerate(files):
        if person1 == person2:
            similarity_matrix[index1, index2] = count_total_variants(genotype_variations[index1])
            None
        else:
            total_variants = count_similar_variants(genotype_variations[index1], genotype_variations[index2])
            similarity_matrix[index1, index2] = total_variants

similarity_matrix = similarity_matrix / similarity_matrix.max()

In [14]:
similarity_matrix

array([[1.    , 0.5534],
       [0.5534, 1.    ]])